In [1]:
import importlib

import model
import pan_loader
import base_config
import loss_functions as L

import utils

import torch
import torch.nn as nn
import torch.optim as optim

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"


In [2]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

train_img_dir = val_img_dir 
train_seg_dir = val_seg_dir 
train_ann_json = val_ann_json 

In [3]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)

In [4]:
config = base_config.Config()

In [5]:
train_loader = pan_loader.get_loader(train_img_dir, train_seg_dir, train_ann, config)
val_loader = pan_loader.get_loader(val_img_dir, val_seg_dir, val_ann, config)

In [6]:
net = model.hgmodel()

In [7]:
def set_trainable(module,state):
    for param in module.parameters():
        param.requires_grad = state

set_trainable(net,False)

set_trainable(net.mb0,True)
set_trainable(net.mb1,True)
set_trainable(net.cb,True)

set_trainable(net.iresnet0, True)
set_trainable(net.iresnet1, True)

# for name,child in net.iresnet0.named_children():
#     if name[:-1] == "layer":
#         [set_trainable(s.ignore_filters,True) for s in child[::2]]
#         [set_trainable(s.copy_filters,True) for s in child[::2]]

param_lr = []
param_lr.append({'params': net.mb0.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.mb1.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.cb.parameters(),'lr':1e-3,'momentum':0.9})

# for name,child in net.iresnet0.named_children():
#     if name[:-1] == "layer":
#         for s in child[::2]:
#             lr = 0
#             if int(name[-1]) > 0:
#                 lr = 1e-3
#             else:
#                 lr = 0
#             param_lr.append({'params':s.ignore_filters.parameters(),'lr':lr,'momentum':0.9})
#             param_lr.append({'params':s.copy_filters.parameters(),'lr':1e-3,'momentum':0.9})

param_lr = [{'params':net.parameters(),'lr':1e-2, 'momentum':0.9}]
net_size = sum([i.numel() for i in net.parameters()])
trainable_params = filter(lambda p: p.requires_grad, net.parameters())
trainable_size = sum([i.numel() for i in trainable_params])
print(net_size,trainable_size)
print(param_lr)
optimizer = optim.SGD(param_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=100, min_lr=1e-2, eps=1e-08)

69912050 69912050
[{'params': <generator object Module.parameters at 0x7f386e9567d8>, 'lr': 0.01, 'momentum': 0.9}]


In [8]:
# net = nn.DataParallel(net, device_ids=[0,1])
net = net.cuda()

In [9]:
ckpt = utils.Checkpoint(iters_per_epoch=60, model_dir="./models/", model_name="first")

for i, data in enumerate(train_loader,0):
#     print("batch %d:"%i)
    optimizer.zero_grad()
    
    images, impulses, instance_masks, cat_ids = utils.cudify_data(data)
    print(cat_ids)
    del(data)
    outs = net([images,impulses])
    del(images, impulses)
    loss = L.loss_criterion1(outs, [instance_masks, cat_ids])
    del(instance_masks, cat_ids, outs)
    ckpt.update(loss.data, net)
    loss.backward()
    del(loss)
    optimizer.step()

[0]
[  1  28 119 132] [1]
[3]
[  1  29  88 121 133] [121]
[4]
[  1   1   1   5 116] [116]
[2]
[  1  38 104] [104]
[3]
[ 75  92 117 120] [120]
[6]
[  1  40  41  46  73  85 121 123 132] [121]
[5]
[  1   1  37 110 116 124 126 132] [124]
[11]
[  1   1   1  55  58  58  63  74  75  88 115 119 132] [119]
[6]
[  1   1  35  91  93  98 120] [120]
[tensor([120], device='cuda:0')]
torch.Size([1, 1, 448, 448]) torch.Size([1, 1, 448, 448]) torch.Size([]) torch.Size([1, 134])
[6]
[  1  34  83  89 100 117 120 126 130] [120]


RuntimeError: dimension specified as 0 but tensor has no dimensions

[8]
[  1   1   1   1   1   1   7  99 113] [113]
[4]
[ 24  24 103 117 120 126] [120]
[6]
[  1   1  38  38 103 104 120] [120]
[4]
[  1   1  64  64  64  65  67  74  74  74  74  74  74  74  74  74  74  82
 105 121 122 128 132] [64]
[0]
[  1   1  44  45  45  56  56  61  88  90  93 105 113 121 122 129 132] [1]
[15]
[  1   1   1   1   1   1   1   1   1   1   1   1   1   5 117 120 124 126
 130] [120]
[18]
[  1   1   1   1   1   1   1   1   1   1   1   1   1  38  38 103 104 117
 120 125] [120]
[13]
[  1   1   1   1   1   1   1   1   1   1   1   6   6   6  10  10  10  10
  13  27  27  92 101 118 120 124 128 130] [6]
[18]
[ 42  42  42  44  44  44  44  44  45  45  70  72  73  84  85  87  88 119
 121 128 132 133] [121]
[26]
[  1   1   1   1   1  27  57  57  57  57  57  57  57  57  57  57  57  57
  66  66  66  66  87 113 119 132 133] [133]
[21]
[  1   1   1   1   1   1  40  40  40  42  42  42  42  45  45  45  45  45
  46  54  57  61  94 122 129 132] [61]
[3]
[  1   1   1   1   1  40  40  40  40  40 